In [6]:
from data_visualization import create_horizontal_bar_chart
from leaderboard_utils import get_mario_leaderboard
import matplotlib.pyplot as plt
import json

# Load the rank data
with open('rank_data_03_25_2025.json', 'r') as f:
    rank_data = json.load(f)

# Get the leaderboard data
df = get_mario_leaderboard(rank_data)

# Create the bar chart
game_name = "Super Mario Bros"
fig = create_horizontal_bar_chart(df, game_name)

# Save the figure in current directory
fig.savefig('mario_performance.png', bbox_inches='tight', dpi=300)

# Display the chart
plt.show()

C:\Users\yuxua\AppData\Local\Temp\ipykernel_14096\1296094769.py:21: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [17]:
from leaderboard_utils import get_combined_leaderboard, GAME_ORDER
from data_visualization import create_radar_charts  # to access the normalization function
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json

# Load the rank data
with open('rank_data_03_25_2025.json', 'r') as f:
    rank_data = json.load(f)

# Get combined leaderboard with all games
selected_games = {game: True for game in GAME_ORDER}
df = get_combined_leaderboard(rank_data, selected_games)

# Create figure and axis with better styling
sns.set_style("whitegrid")
plt.figure(figsize=(16, 9))
ax = plt.gca()

# Get unique models
models = df['Player'].unique()
n_games = len(GAME_ORDER)

# Define a professional color palette
colors = ['#2E86C1', '#E74C3C', '#27AE60', '#F39C12', '#8E44AD',
          '#16A085', '#D35400', '#2980B9', '#C0392B', '#2ECC71']

def normalize_values(values, mean, std):
    """
    Normalize values using z-score and scale to 0-100 range
    """
    if std == 0:
        return [50 if v > 0 else 0 for v in values]  # Handle zero std case
    z_scores = [(v - mean) / std for v in values]
    # Scale z-scores to 0-100 range, with mean at 50
    scaled_values = [max(0, min(100, (z * 30) + 50)) for z in z_scores]
    return scaled_values

# Calculate normalized scores for each game
for game_idx, game in enumerate(GAME_ORDER):
    # Get all scores for this game
    game_scores = []
    for model in models:
        score = df[df['Player'] == model][f'{game} Score'].values[0]
        if score != '_' and float(score) > 0:  # Only include non-zero scores
            game_scores.append((model, float(score)))
    
    # Sort scores from highest to lowest
    game_scores.sort(key=lambda x: x[1], reverse=True)
    
    # Extract sorted models and scores
    sorted_models = [x[0] for x in game_scores]
    scores = [x[1] for x in game_scores]
    
    # Calculate mean and std for normalization
    mean = np.mean(scores)
    std = np.std(scores)
    
    # Normalize scores using the function from data_visualization
    normalized_scores = normalize_values(scores, mean, std)
    
    # Calculate bar width based on number of models in this game
    n_models_in_game = len(sorted_models)
    bar_width = 0.8 / n_models_in_game
    
    # Plot bars for each model
    for i, (model, score) in enumerate(zip(sorted_models, normalized_scores)):
        ax.bar(game_idx + i*bar_width, score, 
              width=bar_width, 
              label=model if game_idx == 0 else "",  # Only add label for first game
              color=colors[i % len(colors)],
              alpha=0.8)

# Customize the plot
ax.set_xticks(np.arange(n_games))
ax.set_xticklabels(GAME_ORDER, rotation=45, ha='right', fontsize=10)
ax.set_ylabel('Normalized Performance Score', fontsize=12)
ax.set_title('AI Model Performance Comparison Across Gaming Tasks\n(Showing only participating models for each game)', 
             fontsize=14, pad=20)

# Add grid lines
ax.grid(True, axis='y', linestyle='--', alpha=0.3)

# Create legend with unique entries
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), 
          bbox_to_anchor=(1.05, 1), 
          loc='upper left',
          fontsize=9,
          title='AI Models',
          title_fontsize=10)

# Add a note about the visualization
plt.figtext(0.5, 0.01, 
           'Note: Scores are normalized within each game. Only models that participated in each game are shown.',
           ha='center', fontsize=9, style='italic')

# Adjust layout to prevent label cutoff
plt.tight_layout()

# Save the figure
plt.savefig('ai_model_gaming_performance_clean.png', bbox_inches='tight', dpi=300)

# Show the plot
plt.show()

C:\Users\yuxua\AppData\Local\Temp\ipykernel_14096\1418582928.py:107: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [31]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from leaderboard_utils import get_combined_leaderboard, GAME_ORDER

# Load the rank data
with open('rank_data_03_25_2025.json', 'r') as f:
    rank_data = json.load(f)

# Load model colors
with open('assets/model_color.json', 'r') as f:
    MODEL_COLORS = json.load(f)

def create_single_radar_chart(df, highlight_models=None):
    # Close any existing figures to prevent memory leaks
    plt.close('all')
    
    # Filter for only the four games we want
    selected_games = ['Super Mario Bros','2048', 'Candy Crash', 'Sokoban']
    game_columns = [f"{game} Score" for game in selected_games]
    categories = selected_games
    
    # Create figure
    fig, ax = plt.subplots(figsize=(12, 8), subplot_kw=dict(projection='polar'))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')
    
    # Compute number of variables
    num_vars = len(categories)
    angles = np.linspace(0, 2*np.pi, num_vars, endpoint=False)
    angles = np.concatenate((angles, [angles[0]]))  # Complete the circle
    
    # Set up the axes
    ax.set_xticks(angles[:-1])
    
    # Format categories with bold text
    formatted_categories = []
    for game in categories:
        if game == "Super Mario Bros":
            game = "Super\nMario"
        elif game == "Candy Crash":
            game = "Candy\nCrash"
        formatted_categories.append(game)
    
    # Set bold labels for categories
    ax.set_xticklabels(formatted_categories, fontsize=10, fontweight='bold')
    
    # Draw grid lines
    ax.set_rgrids([20, 40, 60, 80, 100], 
                  labels=['20', '40', '60', '80', '100'],
                  angle=45,
                  fontsize=8)
    
    # Calculate game statistics for normalization
    def get_game_stats(df, game_col):
        values = []
        for val in df[game_col]:
            if isinstance(val, str) and val == '_':
                values.append(0)
            else:
                try:
                    values.append(float(val))
                except:
                    values.append(0)
        return np.mean(values), np.std(values)
    
    game_stats = {col: get_game_stats(df, col) for col in game_columns}
    
    # Split the dataframe into highlighted and non-highlighted models
    if highlight_models:
        highlighted_df = df[df['Player'].isin(highlight_models)]
        non_highlighted_df = df[~df['Player'].isin(highlight_models)]
    else:
        highlighted_df = pd.DataFrame()
        non_highlighted_df = df
    
    # Plot non-highlighted models first
    for _, row in non_highlighted_df.iterrows():
        values = []
        for col in game_columns:
            val = row[col]
            if isinstance(val, str) and val == '_':
                values.append(0)
            else:
                try:
                    mean, std = game_stats[col]
                    if std == 0:
                        normalized = 50 if float(val) > 0 else 0
                    else:
                        z_score = (float(val) - mean) / std
                        normalized = max(0, min(100, (z_score * 30) + 50))
                    values.append(normalized)
                except:
                    values.append(0)
        
        # Complete the circular plot
        values = np.concatenate((values, [values[0]]))
        
        # Get color for model, use default if not found
        model_name = row['Player']
        color = MODEL_COLORS.get(model_name, '#808080')  # Default to gray if color not found
        
        # Plot with lines and markers
        ax.plot(angles, values, 'o-', linewidth=2, label=model_name, color=color)
        ax.fill(angles, values, alpha=0.25, color=color)
    
    # Plot highlighted models last (so they appear on top)
    for _, row in highlighted_df.iterrows():
        values = []
        for col in game_columns:
            val = row[col]
            if isinstance(val, str) and val == '_':
                values.append(0)
            else:
                try:
                    mean, std = game_stats[col]
                    if std == 0:
                        normalized = 50 if float(val) > 0 else 0
                    else:
                        z_score = (float(val) - mean) / std
                        normalized = max(0, min(100, (z_score * 30) + 30))
                    values.append(normalized)
                except:
                    values.append(0)
        
        # Complete the circular plot
        values = np.concatenate((values, [values[0]]))
        
        # Plot with red color and thicker line
        model_name = row['Player']
        ax.plot(angles, values, 'o-', linewidth=6, label=model_name, color='red')
        ax.fill(angles, values, alpha=0.25, color='red')
    
    # Add title
    plt.title('AI Models Performance Across Classic Games\n(Normalized Scores)',
              pad=20, fontsize=14, fontweight='bold')
    
    # Get handles and labels for legend
    handles, labels = ax.get_legend_handles_labels()
    
    # Reorder legend to put highlighted models first
    if highlight_models:
        highlighted_handles = []
        highlighted_labels = []
        non_highlighted_handles = []
        non_highlighted_labels = []
        
        for handle, label in zip(handles, labels):
            if label in highlight_models:
                highlighted_handles.append(handle)
                highlighted_labels.append(label)
            else:
                non_highlighted_handles.append(handle)
                non_highlighted_labels.append(label)
        
        handles = highlighted_handles + non_highlighted_handles
        labels = highlighted_labels + non_highlighted_labels
    
    # Add legend with reordered handles and labels
    legend = plt.legend(handles, labels,
                       loc='center left',
                       bbox_to_anchor=(1.2, 0.5),
                       fontsize=8,
                       title='AI Models',
                       title_fontsize=10)
    
    # Make the legend title bold
    legend.get_title().set_fontweight('bold')
    
    # Adjust layout to prevent label cutoff
    plt.tight_layout()
    
    return fig

# Get combined leaderboard but only select the four games we want
selected_games = {
    'Super Mario Bros': True,
    'Sokoban': True,
    '2048': True,
    'Candy Crash': True,
    'Tetris (complete)': False,
    'Tetris (planning only)': False
}
df = get_combined_leaderboard(rank_data, selected_games)

# Example usage with highlighted models
highlight_models = ["Llama-4-Maverick-17B-128E-Instruct-FP8"] # Replace with actual model names you want to highlight
fig = create_single_radar_chart(df, highlight_models=highlight_models)
fig.savefig('ai_models_performance_radar_4games.png', bbox_inches='tight', dpi=300)
plt.show()

C:\Users\yuxua\AppData\Local\Temp\ipykernel_7284\1477997120.py:191: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
from leaderboard_utils import get_combined_leaderboard, GAME_ORDER
from data_visualization import create_group_bar_chart
import matplotlib.pyplot as plt
import json

# Load the rank data
with open('rank_data_03_25_2025.json', 'r') as f:
    rank_data = json.load(f)

# Get combined leaderboard with all games
selected_games = {game: True for game in GAME_ORDER}
df = get_combined_leaderboard(rank_data, selected_games)

# Create the group bar chart using the visualization function
# To highlight a specific model, pass its name (or part of its name) to highlight_model
# For example, to highlight Llama-4:
fig = create_group_bar_chart(df, highlight_model="Llama-4")

# Save the figure
plt.savefig('ai_model_gaming_performance_group.png', bbox_inches='tight', dpi=300)

# Show the plot
plt.show() 